## 任务6 - 模型融合（2天）
任务6：模型融合，模型融合方式任意，并结合Task5给出你的最优结果。时间：2天
* 例如Stacking融合，用你目前评分最高的模型作为基准模型，和其他模型进行stacking融合，得到最终模型及评分结果。

[参考答案](./../参考答案)

**Task6（模型融合）** - 模型融合，模型融合方式任意，并结合Task5给出你的最优结果。
例如Stacking融合，用你目前评分最高的模型作为基准模型，和其他模型进行stacking融合，得到最终模型及评分结果。

## 数据集划分

In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 载入数据
with open('final.pkl', 'rb') as f:
    final_data = pickle.load(f)
# data = pd.read_csv("./data.csv",encoding='gbk')
# y=data.status
# 划分训练集测试集
X, y = final_data[final_data.columns.drop("status")], final_data['status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)


# 特征归一化
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

D:\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda2\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


## 模型评估

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score,roc_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline
model_score_train = []
roc_score_train = []
model_score_test = []
roc_score_test = []
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    
    # 准确率
    print(u'[准确率]')
    print(u'训练集：'+ '%.4f'%accuracy_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # 精准率
    print(u'[精准率]')
    print(u'训练集：'+ '%.4f'%precision_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%precision_score(y_test, y_test_pred))

    # 召回率
    print(u'[召回率]')
    print(u'训练集：'+ '%.4f'%recall_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%recall_score(y_test, y_test_pred))
    
    # f1-score
    print(u'[f1-score]')
    print(u'训练集：'+ '%.4f'%f1_score(y_train, y_train_pred))
    print(u'测试集：'+ '%.4f'%f1_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print(u'[auc值]')
    print(u'训练集：'+ '%.4f'%roc_auc_score(y_train, y_train_proba))
    print(u'测试集：'+ '%.4f'%roc_auc_score(y_test, y_test_proba))

## 模型融合 - Stacking

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier

from mlxtend.classifier import StackingClassifier

In [4]:
# 模型调优后得到的参数
lr = LogisticRegression(C = 0.01, penalty = 'l2')
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_poly =  svm.SVC(C = 0.01, kernel = 'poly', probability=True)
svm_rbf =  svm.SVC(gamma = 0.01, C =0.01 , probability=True)
svm_sigmoid =  svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)
dt = DecisionTreeClassifier(max_depth=5,min_samples_split=300,min_samples_leaf=90,max_features=7)
xgb = XGBClassifier(learning_rate =0.01, n_estimators=180, max_depth=3, min_child_weight=5, 
                    gamma=0.4, subsample=0.5, colsample_bytree=0.9, reg_alpha=1, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
lgb = LGBMClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, min_child_weight=11, 
                    gamma=0.1, subsample=0.5, colsample_bytree=0.8, reg_alpha=1e-5, 
                    nthread=4,scale_pos_weight=1, seed=27)

### 调包实现

> 使用4种SVM、决策树、XGBoost和LightGBM作为初级分类器，LR作为次级分类器。

1）将初级分类器产生的类别标签作为新特征

In [5]:
sclf = StackingClassifier(classifiers=[svm_linear, svm_poly, svm_rbf, 
                                       svm_sigmoid, dt, xgb, lgb], meta_classifier=lr)

In [6]:
import warnings
warnings.filterwarnings("ignore")

sclf.fit(X_train, y_train.values)
model_metrics(sclf, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.8188
测试集：0.7821
[精准率]
训练集：0.8387
测试集：0.6644
[召回率]
训练集：0.3429
测试集：0.2702
[f1-score]
训练集：0.4868
测试集：0.3842
[auc值]
训练集：0.6993
测试集：0.6528


2）将初级分类器产生的输出类概率作为新特征

对输出概率use_probas=True，有两种不同的处理方式。

>假设有2个初级分类器和3个类别输出概率：p1=[0.2,0.5,0.3],p2=[0.3,0.4,0.4]。

>如果average_probas=True，则对分类器的结果求平均，得到：p=[0.25,0.45,0.35]

>[推荐]如果average_probas=False，则分类器的所有结果都保留作为新的特征：p=[0.2,0.5,0.3,0.3,0.4,0.4]

In [7]:
sclf = StackingClassifier(classifiers=[svm_linear, svm_poly, svm_rbf, svm_sigmoid, dt, xgb, lgb], 
                            meta_classifier=lr, use_probas=True,average_probas=False)

In [8]:
sclf.fit(X_train, y_train.values)
model_metrics(sclf, X_train, X_test, y_train, y_test)

[准确率]
训练集：0.8130
测试集：0.7807
[精准率]
训练集：0.7944
测试集：0.6554
[召回率]
训练集：0.3429
测试集：0.2702
[f1-score]
训练集：0.4791
测试集：0.3826
[auc值]
训练集：0.8487
测试集：0.7819
